# Fetching Movie Data from Wikipedia

Recommneder systems are one of the most common applications of machine learning. They are usually trained on previously collected ratings from users. To be able to make good suggestions, we need a substantial amount of data per user. Given we don't have it, we'll use outgoing Wikipedia links to collect a dataset of movies that we'll use to train embeddings for both movies and links.

In this notebook, we will implement the required steps to scrap data from Wikipedia dumps. **NOTE**: The final dump is around 13 GB of data and takes a LOT of time to collect, so in the next notebook, focused on the creation of the actual models, we'll use a subset of the top 10.000 movies.

## Before starting...

Let's import the libraries we will use:

In [3]:
import inspect
from helpers import *

ModuleNotFoundError: No module named 'mwparserfromhell'

## Collecting the data

Let's collect the latest dumps from Wikipedia programatically

In [ ]:
print(inspect.getsource())